In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType, StringType, StructField, StructType
from PIL import Image
import mysql.connector

# Define the get_skin_tone_and_undertone function
def get_skin_tone_and_undertone(rgb):
    r, g, b = rgb

    # Define color thresholds for different skin tones
    fair_threshold = (220, 200, 180)
    light_threshold = (200, 180, 150)
    medium_threshold = (180, 150, 120)
    olive_threshold = (160, 130, 100)
    dark_threshold = (140, 100, 80)

    # Compare RGB values with the thresholds
    if r > fair_threshold[0] and g > fair_threshold[1] and b > fair_threshold[2]:
        skin_tone = "Fair"
    elif r > light_threshold[0] and g > light_threshold[1] and b > light_threshold[2]:
        skin_tone = "Light"
    elif r > medium_threshold[0] and g > medium_threshold[1] and b > medium_threshold[2]:
        skin_tone = "Medium"
    elif r > olive_threshold[0] and g > olive_threshold[1] and b > olive_threshold[2]:
        skin_tone = "Olive"
    else:
        skin_tone = "Dark"

    # Determine undertone based on the green and blue channels
    if g > b:
        undertone = "Warm"
    elif g < b:
        undertone = "Cool"
    else:
        undertone = "Neutral"

    return skin_tone, undertone

# Define the extract_rgb UDF
def extract_rgb(image_data):
    image = Image.fromarray(image_data)
    rgb_array = list(image.getdata())
    return rgb_array

# Define the classify_skin_tone UDF
def classify_skin_tone(rgb_array):
    skin_tone, undertone = get_skin_tone_and_undertone(rgb_array)
    return skin_tone, undertone

# Define the MySQL connection parameters
mysql_host = 'localhost'
mysql_database = 'tintandtone'
mysql_user = 'admin'
mysql_password = 'admin'

# Set up the Spark session
spark = SparkSession.builder.appName("SkinToneClassification").getOrCreate()

# Register the UDFs
spark.udf.register("extract_rgb_udf", extract_rgb, ArrayType(IntegerType()))
spark.udf.register("classify_skin_tone_udf", classify_skin_tone, StructType([
    StructField("skinTone", StringType(), nullable=False),
    StructField("undertone", StringType(), nullable=False)
]))

# Load the face photo into Spark
image_path = "./photo-example.jpg"
image_df = spark.read.format("image").load(image_path)

# Extract RGB values
rgb_df = image_df.withColumn("rgb", udf(lambda x: x.tolist(), ArrayType(IntegerType()))("image.data"))

# Classify skin tone and undertone
classified_df = rgb_df.withColumn("skinToneUndertone", udf(lambda x: classify_skin_tone(x), StructType([
    StructField("skinTone", StringType(), nullable=False),
    StructField("undertone", StringType(), nullable=False)
]))("rgb"))

# Save results to MySQL
classified_df.write.format("jdbc").options(
    url=f"jdbc:mysql://{mysql_host}:3306/{mysql_database}",
    driver="com.mysql.cj.jdbc.Driver",
    dbtable="skin",
    user=mysql_user,
    password=mysql_password
).mode("overwrite").save()